In [ ]:
Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [91]:
pickle_file = 'notMNIST.pickle'
#pickle_file = 'notMNIST_sanitized.pickle'


with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [96]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Problem 1a: Logistic Network with L2 Regularization

In [30]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

penalty = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
    + penalty * tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [31]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases.
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 47.234146
Training accuracy: 9.6%
Validation accuracy: 12.2%
Loss at step 100: 11.987520
Training accuracy: 72.7%
Validation accuracy: 70.4%
Loss at step 200: 4.542205
Training accuracy: 77.8%
Validation accuracy: 75.4%
Loss at step 300: 2.016186
Training accuracy: 81.2%
Validation accuracy: 78.5%
Loss at step 400: 1.158716
Training accuracy: 82.9%
Validation accuracy: 80.5%
Loss at step 500: 0.864322
Training accuracy: 83.4%
Validation accuracy: 81.0%
Loss at step 600: 0.762033
Training accuracy: 83.6%
Validation accuracy: 81.3%
Loss at step 700: 0.726156
Training accuracy: 83.5%
Validation accuracy: 81.3%
Loss at step 800: 0.713477
Training accuracy: 83.4%
Validation accuracy: 81.3%
Loss at step 900: 0.708965
Training accuracy: 83.5%
Validation accuracy: 81.2%
Loss at step 1000: 0.707346
Training accuracy: 83.5%
Validation accuracy: 81.2%
Test accuracy: 87.4%


## Problem 1b NN with 1 hidden layer and L2 regularization

In [110]:
batch_size = 128
num_labels = 10
hidden_nodes = 2048

# L2 regularization penalty coeffs
beta_1 = 0.0
beta_2 = 0.0

dropout = True

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  b1 = tf.Variable(tf.zeros([hidden_nodes]))

  w2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
    
  # dropout: probability placeholder - neuron's output kept during training only
  keep_prob = tf.placeholder(tf.float32)

  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  hidden_drop = tf.nn.dropout(hidden, keep_prob)
  if dropout:
    logits = tf.matmul(hidden_drop, w2) + b2
  else:
    logits = tf.matmul(hidden, w2) + b2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
    + beta_1 * tf.nn.l2_loss(w1) + beta_2 * tf.nn.l2_loss(w2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [112]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 669.739502
Minibatch accuracy: 10.2%
Validation accuracy: 25.5%
Minibatch loss at step 500: 68.671959
Minibatch accuracy: 77.3%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 45.814480
Minibatch accuracy: 83.6%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 36.702560
Minibatch accuracy: 78.1%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 11.602568
Minibatch accuracy: 83.6%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 14.337191
Minibatch accuracy: 78.9%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 16.742966
Minibatch accuracy: 77.3%
Validation accuracy: 83.0%
Test accuracy: 89.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

Don't use L2 Regularization and reduce training dataset!

The Minibatch accuracy is close to 100%, but the test accuracy is a lot lower!

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Result: The test accuracy is increased about 8% (~ 70% -> 78%)

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [1]:
batch_size = 256
num_labels = 10
hidden_nodes_1 = 1024
hidden_nodes_2 = 768
hidden_nodes_3 = 256

# L2 regularization penalty coeffs for countering overfitting
beta = 1e-5

# Dropout probability for countering overfitting
dropout = [1.0, 1.0, 1.0]

# exponential decay learning params
exp_decay = True
init_learn_rate = 0.5
decay_steps = 1000
decay_rate = 0.98

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=tf.sqrt(2./(image_size * image_size))))
  b1 = tf.Variable(tf.zeros([hidden_nodes_1]))

  w2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=tf.sqrt(2./(hidden_nodes_1))))
  b2 = tf.Variable(tf.zeros([hidden_nodes_2]))
  
  w3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=tf.sqrt(2./(hidden_nodes_2))))
  b3 = tf.Variable(tf.zeros([hidden_nodes_3]))

  w4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, num_labels], stddev=tf.sqrt(2./(hidden_nodes_3))))
  b4 = tf.Variable(tf.zeros([num_labels]))
    
  # dropout: probability placeholder - neuron's output kept during training only
  keep_prob = tf.placeholder(tf.float32, shape=3)

  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  hidden_2 = tf.nn.relu(tf.matmul(tf.nn.dropout(hidden_1, keep_prob[0]), w2) + b2)
  hidden_3 = tf.nn.relu(tf.matmul(tf.nn.dropout(hidden_2, keep_prob[1]), w3) + b3)
  logits = tf.matmul(tf.nn.dropout(hidden_3, keep_prob[2]), w4) + b4
  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))\
    + beta * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3)) + beta_4 * tf.nn.l2_loss(w4)
  
  # Optimizer.
  if exp_decay:
      global_step = tf.Variable(0)
      learning_rate = tf.train.exponential_decay(init_learn_rate, global_step, decay_steps, decay_rate)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  else:
      optimizer = tf.train.GradientDescentOptimizer(init_learn_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(
              tf.nn.relu(tf.matmul(
                  tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2), w3) + b3), w4) + b4))
  test_prediction = tf.nn.softmax(
      tf.nn.relu(tf.matmul(
          tf.nn.relu(tf.matmul(
              tf.nn.relu(tf.matmul(
                  tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2), w3) + b3), w4) + b4))

NameError: name 'tf' is not defined

In [150]:
import time

num_steps = 10001

with tf.Session(graph=graph) as session:
  start = time.time()
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: dropout}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % (num_steps / 10) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  
  print("Total loop time: <", int(time.time() - start), "secs> or <", int(time.time() - start)
      / 60, "mins and", int(time.time() - start) % 60, "secs>")

Initialized
Minibatch loss at step 0: 2.407238
Minibatch accuracy: 8.6%
Validation accuracy: 28.1%
Minibatch loss at step 1000: 0.469946
Minibatch accuracy: 87.1%
Validation accuracy: 88.5%
Minibatch loss at step 2000: 0.230865
Minibatch accuracy: 92.2%
Validation accuracy: 90.3%
Minibatch loss at step 3000: 0.209224
Minibatch accuracy: 93.0%
Validation accuracy: 90.5%
Minibatch loss at step 4000: 0.162291
Minibatch accuracy: 95.3%
Validation accuracy: 90.7%
Minibatch loss at step 5000: 0.131679
Minibatch accuracy: 95.7%
Validation accuracy: 91.0%
Minibatch loss at step 6000: 0.141924
Minibatch accuracy: 96.5%
Validation accuracy: 90.3%
Minibatch loss at step 7000: 0.127202
Minibatch accuracy: 96.1%
Validation accuracy: 90.8%
Minibatch loss at step 8000: 0.087947
Minibatch accuracy: 98.0%
Validation accuracy: 90.9%
Minibatch loss at step 9000: 0.085421
Minibatch accuracy: 98.4%
Validation accuracy: 91.2%
Minibatch loss at step 10000: 0.050504
Minibatch accuracy: 99.2%
Validation accura